In [1]:
import pandas as pd
import urllib.request
#-------------------------------------------------------------------------------
from google.colab import drive # somente necessário pra uso no google colab
drive.mount('/content/drive')
path = '/content/drive/My Drive/Colab Notebooks/covid19/brasil io/' # DEVE SER MUDADO DE ACORDO COM A APLICAÇÃO (AFETA TODO O RESTO DO CÓDIGO)
# pode ser o caminho raíz do seu projeto
#-------------------------------------------------------------------------------

# PREPARAÇÃO DO DATAFRAME DE REFERÊNCIA 

# carrega os dados do brasil.io
def load_data(date=False):
    url = "https://brasil.io/dataset/covid19/caso?format=csv"
    df  = pd.read_csv(url)
    df  = df.fillna(0)
    df  = df.loc[df["city"] != "Importados/Indefinidos"] # retira dados importados/indefinidos
    if date == False:
        pass
    else:
        df  = df.loc[df["date"] == date]
    return df

# carrega os dados de um arquivo raw do github que contém todas as cidades brasileiras
def cidades():
    url_cidades = "https://raw.githubusercontent.com/leo-smi/latitude-e-longitude-estados-e-cidades-brasileiras/master/cidades.csv"
    df_cidades  = pd.read_csv(url_cidades)
    return df_cidades

# teste de aquisição do dataframe
df = load_data(date = False)
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2020-04-08,AC,Acrelândia,city,9,0.0,True,15256.0,1200013.0,58.99318,0.0000
1,2020-04-08,AC,Bujari,city,1,0.0,True,10266.0,1200138.0,9.74089,0.0000
2,2020-04-08,AC,Plácido de Castro,city,2,0.0,True,19761.0,1200385.0,10.12095,0.0000
3,2020-04-08,AC,Porto Acre,city,1,0.0,True,18504.0,1200807.0,5.40424,0.0000
4,2020-04-08,AC,Rio Branco,city,45,1.0,True,407319.0,1200401.0,11.04785,0.0222
...,...,...,...,...,...,...,...,...,...,...,...
8665,2020-02-27,SP,São Paulo,city,1,0.0,False,12252023.0,3550308.0,0.00816,0.0000
8666,2020-02-27,SP,0,state,1,0.0,False,45919049.0,35.0,0.00218,0.0000
8667,2020-02-26,SP,0,state,1,0.0,False,45919049.0,35.0,0.00218,0.0000
8668,2020-02-25,SP,São Paulo,city,1,0.0,False,12252023.0,3550308.0,0.00816,0.0000


In [13]:
# CORREÇÃO DIÁRIA DO DATASET TANTO PROS ESTADOS QUANTO PARA OS MUNICÍPIOS (ESSA ETAPA DEMANDA TEMPO)
# ESTA CÉLULA SÓ DEVE SER EXECUTADA A PRIMEIRA VE E QUANDO OS DADOS ESTIVEREM DESATUALIZADOS 

# carrega o dataset do brasil.io (exceto os casos indefinidos/importados)
DF = load_data()
# cria uma lista com todos os dias desde o primeiro caso
DATAS = DF.sort_values(by='date', ascending=True).drop_duplicates(subset = 'date')['date'].to_list()
# print(DATAS)
# cria dataframes separados por estados e cidades
DF_estados = DF.loc[DF['place_type'] == 'state']
DF_cidades = DF.loc[DF['place_type'] == 'city']
# cria listas com todos os estados e cidades com casos até o momento
estados = DF_estados.drop_duplicates(subset = ['state']).state.to_list()
cidades = DF_cidades.drop_duplicates(subset = ['city']).city.to_list()


#-------------------------------------------------------------------------------
# CORREÇÃO DOS DADOS ESTADUAIS
#-------------------------------------------------------------------------------
# verifica os dias em que os casos não foram computados nos estados
for i in range(len(estados)):
    estado                  = estados[i] # estado em estados
    datas_estado_real       = DF_estados.loc[DF_estados['state'] == estado].date.to_list() # datas que possuem dados do estado
    primeiro_dia_estado     = min(datas_estado_real) # primeiro em que foi publicado boletim
    ultimo_dia_estado       = max(datas_estado_real) # ultimo dia que os estado publicou boletim
    ultimo_dia_estado_ideal = max(DATAS) # ultimo dia com casos no dataset
    datas_estado_ideal      = pd.date_range(start = primeiro_dia_estado, end = ultimo_dia_estado_ideal, freq='d').strftime('%Y-%m-%d').to_list()#[0].tolist()
    dias_faltantes_estado   = sorted(list(set(datas_estado_ideal) - set(datas_estado_real))) # resta somente os dias que não foram publicados boletins
    # print(f'Estado: {estados[i]},  Primeiro dia: {primeiro_dia_estado},  Ultimo dia: {ultimo_dia_estado},  Dias faltantes: {dias_faltantes_estado}')

# lista de dataframes dos estados por dia
DFs_estados = [] 
for i in range(len(DATAS)):
    DFs_estados.append(DF_estados.loc[DF_estados['date'] == DATAS[i]])

# preenchendo os dias faltantes dos estados, é preenchido os dias faltantes com 
# o valor do dia anterior se o dia em análise é faltante ou o valo é menor que o dia anterior
for i in range(len(DFs_estados)): 
    if i <= len(DFs_estados)-2:
        DFs_estados[i+1] = pd.concat([DFs_estados[i+1], DFs_estados[i]])
        DFs_estados[i+1] = DFs_estados[i+1].sort_values(by=['state', 'confirmed'], ascending=[True, False])
        DFs_estados[i+1] = DFs_estados[i+1].reset_index(drop=True)
        DFs_estados[i+1] = DFs_estados[i+1].drop_duplicates(subset='state', keep='first')
        DFs_estados[i+1] = DFs_estados[i+1].sort_values(by='confirmed', ascending=False)
        DFs_estados[i+1] = DFs_estados[i+1].reset_index(drop=True)
    # corrigir as datas de cada dataframe
    for index, row in DFs_estados[i].iterrows():
        DFs_estados[i]['date'][index] = DATAS[i]

# concatenando todos os dataframes dos estados por dia
dfs_estados = pd.concat(DFs_estados)
dfs_estados = dfs_estados.sort_values(by=['date', 'state'], ascending=[True, True])
dfs_estados = dfs_estados.reset_index(drop=True)
# dfs_estados.head(27)
# dfs_estados = dfs_estados.loc[dfs_estados['state'] == "PR"]    
# dfs_estados.plot('date', 'confirmed')


#-------------------------------------------------------------------------------
# CORREÇÃO DOS DADOS MUNICIPAIS
#-------------------------------------------------------------------------------
# verifica os dias em que os casos não foram computados nas cidades
for i in range(len(cidades)):
    cidade                  = cidades[i] # cidade em cidades
    datas_cidade_real       = DF_cidades.loc[DF_cidades['city'] == cidade].date.to_list() # datas que possuem dados da cidade
    primeiro_dia_cidade     = min(datas_cidade_real) # primeiro em que foi publicado boletim
    ultimo_dia_cidade       = max(datas_cidade_real) # ultimo dia que a cidade publicou boletim
    ultimo_dia_cidade_ideal = max(DATAS) # ultimo dia com casos no dataset
    datas_cidade_ideal      = pd.date_range(start = primeiro_dia_cidade, end = ultimo_dia_cidade_ideal, freq='d').strftime('%Y-%m-%d').to_list()
    dias_faltantes_cidade   = sorted(list(set(datas_cidade_ideal) - set(datas_cidade_real))) # resta somente os dias que não foram publicados boletins
    # print(f'Cidade: {cidades[i]},   Primeiro dia: {primeiro_dia_cidade},   Ultimo dia: {ultimo_dia_cidade},   Dias faltantes: {dias_faltantes_cidade}')

# lista de dataframes dos municipios por dia
DFs_cidades = [] 
for i in range(len(DATAS)):
    DFs_cidades.append(DF_cidades.loc[DF_cidades['date'] == DATAS[i]])

# preenchendo os dias faltantes dos estados
for i in range(len(DFs_cidades)): 
    if i <= len(DFs_cidades)-2:
        DFs_cidades[i+1] = pd.concat([DFs_cidades[i+1], DFs_cidades[i]])
        DFs_cidades[i+1] = DFs_cidades[i+1].sort_values(by=['city', 'confirmed'], ascending=[True, False])
        DFs_cidades[i+1] = DFs_cidades[i+1].reset_index(drop=True)
        DFs_cidades[i+1] = DFs_cidades[i+1].drop_duplicates(subset='city', keep='first')
        DFs_cidades[i+1] = DFs_cidades[i+1].sort_values(by='confirmed', ascending=False)
        DFs_cidades[i+1] = DFs_cidades[i+1].reset_index(drop=True)
    # corrigir as datas de cada dataframe
    for index, row in DFs_cidades[i].iterrows():
        DFs_cidades[i]['date'][index] = DATAS[i]

# concatenando todos os dataframes das cidades por dia
dfs_cidades = pd.concat(DFs_cidades)
dfs_cidades = dfs_cidades.sort_values(by=['date', 'city'], ascending=[True, True])
dfs_cidades = dfs_cidades.reset_index(drop=True)
# dfs_cidades.head(27)
# dfs_cidades = dfs_cidades.loc[dfs_cidades['city'] == "São Paulo"]    
# dfs_cidades.plot('date', 'confirmed', figsize=(20, 5), title='São Paulo')


# CONCATENANDO OS DATAFRAMES DOS ESTADOS E DAS CIDADES
df_FullData = pd.concat([dfs_estados, dfs_cidades])
df_FullData = df_FullData.reset_index(drop=True)


#-------------------------------------------------------------------------------
# ADICIONANDO LATITUDE E LONGITUDE NO DATASET E SALVAMENTO DO MESMO
#-------------------------------------------------------------------------------

try:
    import geocoder # biblioteca geocoder (neste caso específico utiliza a API do bing maps)
except:
    !pip install geocoder
    import geocoder

# esse token é pessoal, mas pode ser usado para testes, em caso de aplicação deve 
# ser gerado um novo com sua conta no link https://www.bingmapsportal.com/Application
key = 'AiXNSTykHCrpomqs090zP_tNFDTuLzim-yg78FJ4xbUKOnFJ7EAUmjIAVN8QGQ1d' 

# Carrega todo o dataset
df_fullData = df_FullData

# Vetor lagitude e longitude que será aplicado como colunas no dataframa
latlong = []

# Dicionário para armazenamento temporário de cidades/estados já analisados (evitar retrabalho de requisição do geocoder)
cidEst = {"cid_est":[], "lat":[], "lng":[]}

# loop que adiciona as latitudes e longitudes
for index, row in df_fullData.iterrows():
	if row["place_type"] == "state":
		cidest = row["state"]
	elif row["place_type"] == "city":
		cidest = row["city"]
		
	if cidest in cidEst["cid_est"]:
		lat = cidEst["lat"][cidEst["cid_est"].index(cidest)]
		lng = cidEst["lng"][cidEst["cid_est"].index(cidest)]
	else:
		loc = None
		while loc is None:
			try:
				if row["place_type"] == "state":
					loc = geocoder.bing(str(cidest) + ', ' + "BR", key=key)
				elif row["place_type"] == "city":
					loc = geocoder.bing(str(cidest)+', '+str(row['state'])+', '+"BR", key=key)
				lat = loc.json["lat"]
				lng = loc.json["lng"]
				cidEst["lat"] = lat
				cidEst["lng"] = lng
				# print(f"{cidest}: {lat, lng}")
			except:
				print(f"{cidest}: CIDADE/ESTADO NÃO ENCONTRADO")
				print("Tentando novamente!")		
		# print("Deu tudo certo!")

	latlong.append([lat, lng])


print(len(latlong))

df_latLong_full = pd.DataFrame(latlong, columns=['latitude', 'longitude'], index=df_fullData.index)

df_fullData_new = pd.concat([df_fullData, df_latLong_full], axis=1)
#-------------------------------------------------------------------------------

# SALVANDO O DATAFRAME 
df_fullData_new.to_csv(path + 'FullData.csv', index=False)

df_fullData_new

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:84: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



     |████████████████████████████████| 102kB 2.7MB/s 
8948


,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate,latitude,longitude
0,2020-02-25,SP,0,state,1,0.0,False,45919049.0,35.0,0.00218,0.0,-23.533970,-46.628868
1,2020-02-26,SP,0,state,1,0.0,False,45919049.0,35.0,0.00218,0.0,-23.533970,-46.628868
2,2020-02-27,SP,0,state,1,0.0,False,45919049.0,35.0,0.00218,0.0,-23.533970,-46.628868
3,2020-02-28,SP,0,state,2,0.0,False,45919049.0,35.0,0.00436,0.0,-23.533970,-46.628868
4,2020-02-29,SP,0,state,2,0.0,False,45919049.0,35.0,0.00436,0.0,-23.533970,-46.628868
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8943,2020-04-08,SP,Votuporanga,city,3,0.0,True,94547.0,3557105.0,3.17303,0.0,-20.421089,-49.975189
8944,2020-04-08,MT,Várzea Grande,city,6,0.0,True,284971.0,5108402.0,2.10548,0.0,-15.651750,-56.139301
8945,2020-04-08,GO,Águas Lindas de Goiás,city,1,0.0,True,212440.0,5200258.0,0.47072,0.0,-15.732810,-48.267769
8946,2020-04-08,SC,Águas Mornas,city,1,0.0,True,6469.0,4200606.0,15.45834,0.0,-27.695589,-48.823269


In [10]:
# TROCA DAS KEYS DAS COLUNAS

import pandas as pd

# Carrega todo o dataset "corrigido"
def loadFullData(sort_by=None):
    df  = pd.read_csv(path + 'FullData.csv')
    df  = df.fillna(0)
    df  = pd.DataFrame({
        'date':df.date,
        'state':df.state,
        'city':df.city,
        'level':df.place_type,
        'lat':df.latitude,
        'lng':df.longitude,
        'cases':df.confirmed,
        # 'isLast':df.is_last, # não é necessário de foi corrigido as lacunas do dataset
        'pop2019':df.estimated_population_2019,
        'ibgeCode':df.city_ibge_code,
        'casesPer100k':df.confirmed_per_100k_inhabitants,
        'deaths':df.deaths,
        'deathRate':df.death_rate,
    })
    return df

loadFullData()

,date,state,city,level,lat,lng,cases,pop2019,ibgeCode,casesPer100k,deaths,deathRate
0,2020-02-25,SP,0,state,-23.533970,-46.628868,1,45919049.0,35.0,0.00218,0.0,0.0
1,2020-02-26,SP,0,state,-23.533970,-46.628868,1,45919049.0,35.0,0.00218,0.0,0.0
2,2020-02-27,SP,0,state,-23.533970,-46.628868,1,45919049.0,35.0,0.00218,0.0,0.0
3,2020-02-28,SP,0,state,-23.533970,-46.628868,2,45919049.0,35.0,0.00436,0.0,0.0
4,2020-02-29,SP,0,state,-23.533970,-46.628868,2,45919049.0,35.0,0.00436,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8943,2020-04-08,SP,Votuporanga,city,-20.421089,-49.975189,3,94547.0,3557105.0,3.17303,0.0,0.0
8944,2020-04-08,MT,Várzea Grande,city,-15.651750,-56.139301,6,284971.0,5108402.0,2.10548,0.0,0.0
8945,2020-04-08,GO,Águas Lindas de Goiás,city,-15.732810,-48.267769,1,212440.0,5200258.0,0.47072,0.0,0.0
8946,2020-04-08,SC,Águas Mornas,city,-27.695589,-48.823269,1,6469.0,4200606.0,15.45834,0.0,0.0


In [11]:
# VISUALIZANDO O DATAFRAME DOS ESTADOS

df = loadFullData()
df = df.loc[df['level'] == 'state']
df

,date,state,city,level,lat,lng,cases,pop2019,ibgeCode,casesPer100k,deaths,deathRate
0,2020-02-25,SP,0,state,-23.533970,-46.628868,1,45919049.0,35.0,0.00218,0.0,0.0000
1,2020-02-26,SP,0,state,-23.533970,-46.628868,1,45919049.0,35.0,0.00218,0.0,0.0000
2,2020-02-27,SP,0,state,-23.533970,-46.628868,1,45919049.0,35.0,0.00218,0.0,0.0000
3,2020-02-28,SP,0,state,-23.533970,-46.628868,2,45919049.0,35.0,0.00436,0.0,0.0000
4,2020-02-29,SP,0,state,-23.533970,-46.628868,2,45919049.0,35.0,0.00436,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...
726,2020-04-08,RS,0,state,-30.039530,-51.216930,555,11377239.0,43.0,4.87816,8.0,0.0144
727,2020-04-08,SC,0,state,-27.599840,-48.541821,501,7164788.0,42.0,6.99253,17.0,0.0339
728,2020-04-08,SE,0,state,-10.910600,-37.059898,39,2298696.0,28.0,1.69661,4.0,0.1026
729,2020-04-08,SP,0,state,-23.533970,-46.628868,6708,45919049.0,35.0,14.60832,428.0,0.0638


In [12]:
# GRÁFICO DE BARRAS DOS CASOS E ÓBITOS ACUMULADOS DE UM DETERMINADO ESTADO

# Ultimo dia
df = loadFullData()
# df = df.loc[df['date'] == max(df['date'])]
df = df.loc[df['level'] == 'state']
df = df.loc[df['state'] == 'SP']
# df = df.loc[df['isLast'] == True]
df = df.sort_values(by='date', ascending=True)
df = df.reset_index(drop=True)

import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Bar(
    x            = df["date"],
    y            = df["cases"],
    name         = 'Casos acumulados',
    marker_color = 'indianred'
))

fig.add_trace(go.Bar(
    x            = df["date"],
    y            = df["deaths"],
    name         = 'Mortes acumuladas',
    marker_color = 'yellow'
))


fig.update_layout(xaxis_tickangle=-90, yaxis_type="log", barmode='group')
fig.show()

In [9]:
# GRÁFICO DE BARRAS DOS CASOS E ÓBITOS ACUMULADOS DE UMA DETERMINADA CIDADE

# Ultimo dia
df = loadFullData()
# df = df.loc[df['date'] == max(df['date'])]
df = df.loc[df['level'] == 'city']
df = df.loc[df['city'] == 'Rio de Janeiro']
# df = df.loc[df['isLast'] == True]
df = df.sort_values(by='date', ascending=True)
df = df.reset_index(drop=True)

import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Bar(
    x            = df["date"],
    y            = df["cases"],
    name         = 'Casos acumulados',
    marker_color = 'indianred'
))

fig.add_trace(go.Bar(
    x            = df["date"],
    y            = df["deaths"],
    name         = 'Mortes acumuladas',
    marker_color = 'yellow'
))


fig.update_layout(xaxis_tickangle=-90, yaxis_type="log", barmode='group')
fig.show()